### setup

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd

import scipy


from utils import *
from pytorch_sparse_utils import *


import torch
from torch import nn
import torch.nn.functional as F

### set parameters

In [2]:
DECOMP_RANK = 10
LAPLACIAN_PARAM = 0.001
TV_PARAM = 0.001

In [3]:
# Parameters
DECOMP_RANK = 100
LAPLACIAN_PARAM = 0.01
TV_PARAM = 0.1


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


### loading data

In [5]:
bead_reads_subset = pd.read_csv('./data/bead_reads_subset.csv', index_col='Unnamed: 0')
bead_reads_subset.head()

,1110008P14Rik,1500009C09Rik,1500012F01Rik,1700020I14Rik,2010107E04Rik,2010300C02Rik,2210016L21Rik,2310036O22Rik,2900011O08Rik,3110035E14Rik,...,Zrsr1,Zwint,mt-Co1,mt-Cytb,mt-Nd1,mt-Nd2,mt-Nd4,mt-Nd5,mt-Rnr1,mt-Rnr2
AAAAAAAGGTAGTA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
AAAAAAAGTCCCAA,0,0,0,0,0,0,0,0,0,0,...,0,0,2,2,5,0,1,0,2,4
AAAAAAATCTTAGT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAAAAACATCTTTC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
AAAAAACGAAATAG,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [6]:
data_scipy_sparse = scipy.sparse.coo_matrix( bead_reads_subset.to_numpy() )

In [7]:
data_sparse_tensor = scipy_sparse_to_pytorch_sparse(data_scipy_sparse).to(device=device)

In [8]:
data_sparse_tensor

tensor(indices=tensor([[    0,     0,     0,  ..., 53207, 53207, 53207],
                       [   84,   412,   528,  ...,  1219,  1220,  1221]]),
       values=tensor([ 1.,  1.,  1.,  ..., 12., 26., 71.]),
       device='cuda:0', size=(53208, 1222), nnz=9552845, layout=torch.sparse_coo)

In [9]:
data_dense_tensor = data_sparse_tensor.to_dense()

In [10]:
mask_scipy_sparse = (data_scipy_sparse != 0).astype(np.int64).tocoo()

In [11]:
mask_sparse_tensor = scipy_sparse_to_pytorch_sparse(mask_scipy_sparse)

In [12]:
mask_dense_tensor = mask_sparse_tensor.to_dense().bool()

In [13]:
distances = np.load('./data/macosko_distance_matrix.npy')

In [14]:
adjacency = scipy.sparse.coo_matrix( distances <= 20 , dtype=np.float32)
degree = scipy.sparse.diags( adjacency.sum(axis=0).A1 )
laplacian = (degree - adjacency).tocoo()

In [15]:
adjacency_sparse_tensor = scipy_sparse_to_pytorch_sparse(adjacency)

In [16]:
adjacency_dense_tensor = adjacency_sparse_tensor.to_dense()

In [17]:
def laplacian_loss(x, x_lap):
    
    lap_sum = 0
    for ii in range(x.shape[1]):
        lap_sum = lap_sum + (x[:,ii].T @ x_lap @ x[:,ii])
        
    return lap_sum

In [18]:
mf = MatrixFactorization(mask_sparse_tensor.shape[0], mask_sparse_tensor.shape[1], DECOMP_RANK).to(device=device)

In [19]:
import torch.optim as optim

optimizer = optim.Adam(mf.parameters(), lr=0.001)

In [20]:
losses = []

for epoch in range(50):
    
    batcher = data_adjacency_batcher(data_dense_tensor, adjacency_dense_tensor, 5000)
    
    total_loss = 0
    
    
    for (data_batch, mask_batch, adjacency_batch, row_batch) in batcher:
        
        
        
        data_batch = data_batch.to(device=device)
        mask_batch = mask_batch.to(device=device)
        adjacency_batch = adjacency_batch.to(device=device)
        row_batch = row_batch.to(device=device)
        
        laplacian_tensor = adjacency_to_laplacian(adjacency_batch).to(device=device)
    
        optimizer.zero_grad()

        output = mf.forward(row_batch, torch.arange(0, data_batch.shape[1]))
        
        L1_laplacian_tensor = adjacency_to_laplacian(adjacency_to_L1_adjacency(output, adjacency_batch, 0.001, device)).to(device=device)
        
        loss = squared_loss(data_batch[mask_batch], output[mask_batch]) + LAPLACIAN_PARAM*laplacian_loss(output, laplacian_tensor) + TV_PARAM*laplacian_loss(output, L1_laplacian_tensor)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        

    if epoch % 1 == 0:
        print(epoch)
        print(total_loss)
        
    losses.append(total_loss)

0
215980302.0


1
215791098.0


2
215300216.0


3
214376334.0


4
212950610.0


...

45
154187500.0


46
153451332.0


47
152730903.0


48
152018427.5


49
151305373.0


In [21]:
loss.item()

12367276.0

In [22]:
np.save(f'./results/loss_curves/mf_rank{DECOMP_RANK}_L{LAPLACIAN_PARAM}_TV{TV_PARAM}_losses', np.array(losses))

In [23]:
torch.save(mf.state_dict(), f'./results/saved_models/mf_rank{DECOMP_RANK}_L{LAPLACIAN_PARAM}_TV{TV_PARAM}_state')

In [24]:
final_output = mf.forward(torch.arange(0, data_dense_tensor.shape[0]), torch.arange(0, data_dense_tensor.shape[1]))

final_squared_error = squared_loss(data_dense_tensor, final_output)
final_laplacian = laplacian_loss(final_output, adjacency_to_laplacian(adjacency_dense_tensor).to(device=device))
final_TV =  laplacian_loss(final_output, adjacency_to_laplacian(adjacency_to_L1_adjacency(final_output, adjacency_dense_tensor, 0.001, device)).to(device=device))

In [25]:
np.save(f'./results/final_losses/mf_rank{DECOMP_RANK}_L{LAPLACIAN_PARAM}_TV{TV_PARAM}_losses', np.array([final_squared_error.item(), final_laplacian.item(), final_TV.item()]))